# Gerenciador de investimentos

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
# from lib import lib as l
from lib import plot as p
from lib import data as d
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#27FEFA'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2', '#162CF2'])

## Leitura dos dados

In [3]:
dinv, dport, dport_inv, dprices = f.read_invest()
dinvest = pd.concat([dinv, dport_inv]).reset_index(drop=True)

## Fluxos de investimentos

### Histórico de fluxos

In [4]:
dflow = dinvest.groupby('ID').apply(f.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow['Rend'] = dflow['Rendimento anual'].apply(d.pct2int)
dflow = dflow.sort_values(by=['Rend'], ascending=False).reset_index(drop=True)
a = dflow.pop('Rend')

### Fluxos ativos

In [5]:
dflow[dflow['Ativo']]

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ganho em 1 ano,Ativo,Tipo
0,100.000,1308.000,219.200,5.377,16.76%,2.92%,41.17%,628.822,True,IVVB11
1,102.000,1390.300,136.900,3.967,9.85%,2.4%,32.75%,500.166,True,IVVB11
2,105.000,1478.000,49.200,2.328,3.33%,1.42%,18.33%,280.001,True,IVVB11
10,14.300,930.000,3.030,1.082,0.33%,0.3%,3.66%,34.179,True,Nu
16,1.400,8952.240,3.070,0.131,0.03%,0.26%,3.18%,284.615,True,Nu
18,13.000,10000.000,42.770,1.738,0.43%,0.25%,2.98%,299.558,True,Nu
19,15.000,3000.000,6.610,0.951,0.22%,0.23%,2.81%,84.451,True,Nu
27,16.000,3189.460,1.270,0.393,0.04%,0.1%,1.22%,38.872,True,Nu
35,109.000,1306.700,0.000,2.295,0.0%,0.0%,0.0%,0.000,True,Cash
36,101.000,875.000,-221.500,4.885,-25.31%,-5.8%,-51.08%,-333.817,True,BRAX11


## Performance

In [6]:
dnu = dflow[dflow['Meses'] > 0]
dnu = dnu[dnu['Tipo'] == 'Nu']

detf = dflow[dflow['Meses'] > 0]
detf = detf[detf['Tipo'] != 'Nu']

### Renda fixa

In [7]:
p.change_df_prop(f.calc_flows_overview(dnu), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
26128.4,300.69,0.25%,741.674,2.84%


### Renda Variável

In [8]:
p.change_df_prop(f.calc_flows_overview(detf), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
9820.1,-1290.3,-11.61%,-1785.89,-18.19%


### Mensal

In [9]:
fig_prof = p.make_fig()
fig_prof_total = p.make_fig()

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [10]:
dret = d.pivot_agg(f.calc_flow_gains(dinvest[dinvest['Tipo'] != 'Cash']), val='ValorDelta', idx='Data', col='Tipo', agg={'ValorDelta':np.sum})
dret['Portfolio'] = dret.sum(axis=1)
cols = dret.columns[1:]
for c in cols:
    dret = d.window(dret, val=c)

In [11]:
palette1 = {'Portfolio': lcolors[0], 'BRAX11': lcolors[2], 'IVVB11': lcolors[0], 'BBSD11': lcolors[1], 'Cash': lcolors[4]}
palette2 = {'PortfolioAvg': dcolors[0], 'BRAX11Avg': dcolors[2], 'IVVB11Avg': dcolors[0], 'BBSD11Avg': dcolors[1], 'Cash': dcolors[4]}
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette1, y=dret.columns[5:6], plot_type='bar')
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette2, y=dret.columns[10:11], name=['Portfolio médio'])

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …